In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
r = requests.get(URL)
soup = BeautifulSoup(r.content)
# Append to the full set of results
results = soup.findAll('div', { "class" : "result" })

/Users/AntonioPasciuto/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/AntonioPasciuto/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
def extract_text(el):
    if el:
        return el.text.strip()
    else:
        return ''
        

def get_company_from_result(result):
    return extract_text(result.find('span', {'class' : 'company'}))

def get_location_from_result(result):
    return  extract_text(result.find('span', {'class' : 'location'}))

def get_summary_from_result(result):
    return  extract_text(result.find('span', {'class' : 'summary'}))

def get_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def get_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        
    return None

In [5]:
YOUR_CITY = 'Atlanta'

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        r = requests.get(url_template.format(city, start))
        # Grab the results from the request (as above)
        soup = BeautifulSoup(r.content)
        # Append to the full set of results
        results += soup.findAll('div', { "class" : "result" })

/Users/AntonioPasciuto/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/AntonioPasciuto/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
rows = []
for result in results:
    if result:
        row = {}
        row['title'] = get_title_from_result(result)
        row['company'] = get_company_from_result(result)
        row['summary'] = get_summary_from_result(result)
        row['salary'] = get_salary_from_result(result)
        row['city'] = city
        rows.append(row)

import pandas as pd
data = pd.DataFrame.from_records(rows)

data.head()

,city,company,salary,summary,title
0,New+York,Predictive Science,None,This is a freelance data scientist position wh...,Data Scientist
1,New+York,Numeric Jobs,None,Manage multiple online & offline data sources ...,Statistical Data Analyst
2,New+York,Pharma Resource Group Inc,None,"Ability to draw conclusions from data, and to ...",Sr. Analytical Scientist
3,New+York,Susquehanna International Group,None,Your work will impact all aspects of the trade...,Quantitative Researcher - Machine Learning
4,New+York,University of Pennsylvania,None,Experience with data science a plus. Work with...,SENIOR DATA SCIENTIST


In [8]:
# Filter to only the rows that have salary entries
data = data[data.salary.notnull()]

# Remove duplicates
data = data.drop_duplicates()

# Filter out salary entries referring to week, hour or month
data = data[~(data.salary.astype('str').str.contains('hr'))]
data = data[~(data.salary.astype('str').str.contains('hour'))]
data = data[~(data.salary.astype('str').str.contains('week'))]
data = data[~(data.salary.astype('str').str.contains('wk'))]
data = data[~(data.salary.astype('str').str.contains('month'))]

In [9]:
import re
import numpy as np
def extract_salary_average(salary_string):
    regex = r'\$([0-9]+,[0-9]+)'
    matches = re.findall(regex, salary_string)
    return np.mean([float(salary.replace(',', '')) for salary in matches ])

In [10]:
data['parsed_salary'] = data['salary'].map(extract_salary_average)